In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LSTM**

In [4]:
import pandas as pd
data = pd.read_csv("sample_data/news.csv")

In [5]:
data.dropna(axis=0, subset=['Article Content','Kategori'], inplace=True)
data.head()

,Nomor,Article Title,Article Link,Article Content,Kategori
0,1,"7 Hobi yang Bisa Datangkan Cuan di 2022, Ada H...",https://www.detik.com/edu/edutainment/d-588006...,Meski banyak orang menganggap hobi hanya sebat...,EDU
1,2,"Perbanyak Ahli Gunung Api, ITN Yogyakarta Buka...",https://www.detik.com/edu/perguruan-tinggi/d-5...,Institut Teknologi Nasional (ITN) Yogyakarta m...,EDU
2,3,"PTM Terbatas 2022 di Jakarta, Ini Aturan Kegia...",https://www.detik.com/edu/sekolah/d-5880075/pt...,Pemerintah Provinsi DKI Jakarta melalui Kepala...,EDU
3,4,Mengapa Perlu Menjaga Kelestarian Tanaman Baka...,https://www.detik.com/edu/detikpedia/d-5879161...,Tanaman bakau adalah salah satu ekosistem yang...,EDU
4,5,Cara Membuat Surat Lamaran Kerja yang Sempurna...,https://www.detik.com/edu/edutainment/d-587992...,Membuat Curriculum Vitae (CV) dan surat lamara...,EDU


In [6]:
data[130:140]

,Nomor,Article Title,Article Link,Article Content,Kategori
130,131,"Surat Az Zukhruf, Firman Allah SWT yang Mencer...",https://www.detik.com/edu/detikpedia/d-5883699...,"""Salah satu faedah surat Az Zukhruf adalah pen...",EDU
131,132,Doa Setelah Membaca Surat Al Waqiah dan Waktu ...,https://www.detik.com/edu/detikpedia/d-5883671...,Bacaan doa setelah membaca surat Al Waqiah ini...,EDU
132,133,"MUI Sudah 46 Tahun Berdiri, Seperti Apa Sejara...",https://www.detik.com/edu/detikpedia/d-5887336...,"Saat ini, Majelis Ulama Indonesia dipimpin ole...",EDU
133,134,Ortu di Jakarta yang Cemas PTM 100 % Diberi Ke...,https://www.detik.com/edu/sekolah/d-5887318/or...,Pembelajaran tatap muka (PTM) 100 persen sudah...,EDU
134,135,"Profil Kazakhstan, Negara Pecahan Uni Soviet d...",https://www.detik.com/edu/detikpedia/d-5887241...,Kazakhstan adalah negara pecahan Uni Soviet da...,EDU
135,136,Sultan Izinkan Sekolah-Kampus Gelar PTM 100 Pe...,https://www.detik.com/edu/perguruan-tinggi/d-5...,Gubernur Daerah Istimewa Yogyakarta Sri Sultan...,EDU
136,137,Ortu Was-was Anak Harus PTM 100 Persen? Psikol...,https://www.detik.com/edu/sekolah/d-5886908/or...,Kebijakan itu menuai pro dan kontra. Orang tua...,EDU
137,138,5 Contoh Teks Berita Dilengkapi Penjelasan dan...,https://www.detik.com/edu/detikpedia/d-5886893...,Kejadian atau peristiwa yang sedang dan telah ...,EDU
138,139,Ini Daftar Pekerjaan yang Bakal Digantikan Rob...,https://www.detik.com/edu/detikpedia/d-5886814...,Beberapa posisi pekerjaan saat ini sudah bisa ...,EDU
139,140,"Mengenal Serat, Bahan Baku Benang dan Kain",https://www.detik.com/edu/detikpedia/d-5886805...,"Mengutip dari Sumber Belajar Kemdikbud, sifat ...",EDU


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train,X_test, y_train, y_test = train_test_split(data['Article Content'],data['Kategori'],train_size=0.8,random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$$%&()*+,-./:;<=>?@[\]^_{|}~', lower=True)
tokenizer.fit_on_texts(data['Article Content'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(data['Article Content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 43383 unique tokens.
Shape of data tensor: (2493, 250)


In [9]:
Y = pd.get_dummies(data['Kategori']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2493, 6)


In [10]:
X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2243, 250) (2243, 6)
(250, 250) (250, 6)


In [11]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/20
32/32 [==============================] - 30s 817ms/step - loss: 1.6854 - accuracy: 0.3632 - val_loss: 1.4152 - val_accuracy: 0.3422
Epoch 2/20
32/32 [==============================] - 26s 815ms/step - loss: 1.1890 - accuracy: 0.4886 - val_loss: 1.2068 - val_accuracy: 0.4667
Epoch 3/20
32/32 [==============================] - 26s 802ms/step - loss: 0.8721 - accuracy: 0.6184 - val_loss: 1.0034 - val_accuracy: 0.6400
Epoch 4/20
32/32 [==============================] - 26s 822ms/step - loss: 0.7403 - accuracy: 0.7636 - val_loss: 1.0595 - val_accuracy: 0.6044
Epoch 5/20
32/32 [==============================] - 26s 829ms/step - loss: 0.6165 - accuracy: 0.8558 - val_loss: 0.8578 - val_accuracy: 0.7467
Epoch 6/20
32/32 [==============================] - 26s 826ms/step - loss: 0.8572 - accuracy: 0.7562 - val_loss: 1.6720 - val_accuracy: 0.3200
Epoch 7/20
32/32 [==============================] - 25s 793ms/step - loss: 0.7179 - accuracy: 0.7587 - val_loss: 0.9662 - val_accuracy: 0.6533

In [12]:
y_pred = model.predict(X_test)
y_pred.shape

(250, 6)